In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import io
from temp import maping_cols
%matplotlib inline
pd.set_option('display.max_columns', None)


In [2]:
url_dados = 'http://dados.prefeitura.sp.gov.br/pt_PT/dataset/aprovacao-de-alvaras'

In [3]:
def pegar_links(url_dados):
    
    with requests.get(url_dados) as r:
        assert r.status_code == 200
        html = r.text
    soup = BeautifulSoup(html)
    recursos = soup.findAll('li', {'class':'resource-item'})
    links = []
    for recurso in recursos:
        link = recurso.find(class_="resource-url-analytics")['href']
        links.append(link)
    return links

In [4]:
def filtrar_por_extensao(links, extensao):
    
    return [link for link in links if link[-4:]==extensao]

In [5]:
def pegar_ano(link):
    
    ano = link.split('/')[-1].split('.')[0]
    ano = ''.join([char for char in ano if char.isdigit()])
    return ano

In [6]:
def ler_csv(link):
    ano = pegar_ano(link)
    
    primeira_col_zuada = {'2012', '2011', '2010',
                          '2009', '2008', '2007',
                          '2006', '2005', '2004',
                          '2002'}
    teste = ano in primeira_col_zuada
    with requests.get(link) as r:
        t = r.text
    file_like = io.StringIO(t)
    if not teste:
        df = pd.read_csv(file_like, sep = ';', dtype = 'object',
                         encoding='utf-8', engine = 'python')
    else:
        df = pd.read_csv(file_like, sep = ';', dtype = 'object',
                         encoding='utf-8', engine = 'python',
                        skiprows = [0])
    
    return df

In [7]:
def clean_cols(df):
    
    df = df.copy()
    
    for col in df.columns:
        
        col_nova = col.lower().strip()
        df.rename({col:col_nova}, axis = 1, inplace = True)
    return df

In [8]:
def padronizar_cols(df, maping_cols):
    
    df = df.copy()
    for col in df.columns:
        for col_padrao, cols_erradas in maping_cols.items():
            if col in cols_erradas:
                df.rename({col : col_padrao}, axis = 1, inplace = True)
    return df

In [9]:
def juntar_categorias_uso(df):
    
    df = df.copy()
    cats = {'catuso2', 'catuso3', 'catuso4'}
    cols = set(df.keys())
    teste = cats.issubset(cols)
    if teste:
    
        df['categoria de uso'] = df.apply(lambda row: '//'.join([
                                        str(row['categoria de uso']),
                                        str(row['catuso2']), 
                                        str(row['catuso3']),
                                        str(row['catuso4'])
                                        ]), axis = 1)
        df.drop(list(cats), axis = 1, inplace = True)
        
    return df

In [10]:
def drop_na_rows(df):
    
    df = df.dropna(axis = 0, how = 'all').copy()
    
    return df

In [11]:
def limpar_cols_unnamed(df):
    
    cols_limpas = [col for col in df.keys() if
                  "Unnamed:" not in col[:8]]
    
    return df[cols_limpas].copy()

In [12]:
def drop_na_cols(df):
    
    df = df.dropna(axis = 1, how = 'all').copy()
    
    return df

In [13]:
def gerar_dfs_csvs(links,maping_cols=maping_cols):
    
    dfs = []
    for link in links:
        df = ler_csv(link)
        df = drop_na_rows(df)
        df = drop_na_cols(df)
        df = limpar_cols_unnamed(df)
        ano = pegar_ano(link)
        df['ano'] = ano
        df = clean_cols(df)
        df = juntar_categorias_uso(df)
        df = padronizar_cols(df, maping_cols)
        dfs.append(df)
    return dfs

In [14]:
def concatenar_dfs(lista_dfs):
    
    return pd.concat(lista_dfs, ignore_index=True, sort = False)

In [15]:
def pipeline_extracao(url_dados):
    
    links = pegar_links(url_dados)
    csvs = filtrar_por_extensao(links, '.csv')
    dfs = gerar_dfs_csvs(csvs)
    df_final = concatenar_dfs(dfs)
    
    return df_final

In [16]:
df = pipeline_extracao(url_dados)

In [17]:
def teste_rows_vazias(df):
    
    anos = df['ano'].unique()
    
    for ano in anos:
        df_ano = df[df['ano']==ano]
        num_rows = len(df_ano)
        rows_vazias = num_rows - len(df_ano.drop(['ano'], axis = 1).dropna(how = 'all'))
        print(ano, ':', rows_vazias)

In [18]:
teste_rows_vazias(df)

2018 : 0
2017 : 0
2016 : 0
2015 : 0
2014 : 0
2013 : 0
2012 : 0
2011 : 0
2010 : 0
2009 : 0
2008 : 0
2007 : 0
2006 : 0
2005 : 0
2004 : 0
2003 : 0
2002 : 0


In [36]:
def col_vazia_por_ano(df):
    '''Testa para ver se existe alguma coluna que é vazia para um ano todo'''
    
    result = {}
    anos = df['ano'].unique()
    for ano in anos:
        df_ano = df[df['ano']==ano]
        
        for col in df.keys():

            qtd_nul = df_ano[col].isnull().sum()
            if qtd_nul == len(df_ano):
                if ano not in result:
                    result[ano] = []
                result[ano].append(col)
    return result

In [38]:
col_vazia_por_ano(df)

{'2018': ['tipo_constru',
  'firma/dirigente tecnico',
  'responsavel tecnico',
  'responsavel da firma',
  'numero de blocos',
  'numero de unidades',
  'numero de pavimentos',
  'areaexist',
  'areaaaum',
  'areaauto',
  'num_endereco',
  'sequencia',
  'ident',
  'coddocu',
  'codadtv'],
 '2017': ['tipo_constru',
  'firma/dirigente tecnico',
  'responsavel tecnico',
  'responsavel da firma',
  'numero de blocos',
  'numero de unidades',
  'numero de pavimentos',
  'areaexist',
  'areaaaum',
  'areaauto',
  'num_endereco',
  'sequencia',
  'ident',
  'coddocu',
  'codadtv'],
 '2016': ['tipo_constru',
  'firma/dirigente tecnico',
  'responsavel tecnico',
  'responsavel da firma',
  'numero de blocos',
  'numero de unidades',
  'numero de pavimentos',
  'areaexist',
  'areaaaum',
  'areaauto',
  'num_endereco',
  'sequencia',
  'ident',
  'coddocu',
  'codadtv'],
 '2015': ['dt_autuacao',
  'tipo_constru',
  'firma/dirigente tecnico',
  'responsavel tecnico',
  'responsavel da firma',
 

In [28]:
df.keys()

Index(['mes', 'unidade', 'subprefeitura', 'alvara', 'proc_num', 'desc_alvara',
       'dt_autuacao', 'sql', 'categoria_uso', 'zoneamento',
       'zona de uso anterior', 'bairro', 'area_constru', 'proprietario',
       'area_terreno', 'endereco', 'dt_aprovacao', 'dirigente técnico',
       'responsável pela empresa', 'autor_proj', 'responsável pela empresa.1',
       'blocos_pavimentos_unidades', 'ano', 'tipo_constru',
       'firma/dirigente tecnico', 'responsavel tecnico',
       'responsavel da firma', 'numero de blocos', 'numero de unidades',
       'numero de pavimentos', 'areaexist', 'areaaaum', 'areaauto',
       'num_endereco', 'sequencia', 'ident', 'coddocu', 'codadtv'],
      dtype='object')

In [29]:
df

,mes,unidade,subprefeitura,alvara,proc_num,desc_alvara,dt_autuacao,sql,categoria_uso,zoneamento,zona de uso anterior,bairro,area_constru,proprietario,area_terreno,endereco,dt_aprovacao,dirigente técnico,responsável pela empresa,autor_proj,responsável pela empresa.1,blocos_pavimentos_unidades,ano,tipo_constru,firma/dirigente tecnico,responsavel tecnico,responsavel da firma,numero de blocos,numero de unidades,numero de pavimentos,areaexist,areaaaum,areaauto,num_endereco,sequencia,ident,coddocu,codadtv
0,Janeiro,SGM/AJ,PENHA,2018.01.369-00,2004-1.010.334-5,RECONSIDERACAO DO DESPACHO DE AUTO DE REGULARI...,15/12/2004,059.268.0042-0,R,NaN,,PENHA,NaN,ESPOLIO DE ABILIO BERNARDO CARDOSO VALENTE E O...,"318,13",R SAIVA 00305,30/01/2018,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Janeiro,SGM/AJ,PINHEIROS,2018.00.491-00,2003-1.037.949-7,RECONSIDERACAO DO DESPACHO DE AUTO DE REGULARI...,22/10/2003,299.133.0004-3,R; E1; S1; C1,NaN,Z9-022.,VILA OLIMPIA,NaN,MARIA CECILIA KALIL BEYRUTI,"400,00",R CHILON 00060,12/01/2018,AIDA MARTINS CASIMIRO VARUZZI,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Janeiro,SMUL,CAMPO LIMPO,2018.00.889-00,2014-0.176.365-7,RECONSIDERACAO DE DESPACHO DE ALVARA DE APROVA...,27/06/2014,169.269.0009-3,EHMP,ZEUP,ZONA DE CENTRALIDADE POLAR - A.,VILA AMERICA,"12.123,70",PROJETO IMOBILIARIO E 22 LTDA,"3.678,24",ES VELHA DE ITAPECERICA 00019,19/01/2018,HERCY MARTINS COSTA,NaN,MARCELO AUGUSTO FERNANDES BARTOLO,NaN,B: 1 P: 16 U: 187; B: 1 P: 2 U: 0,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Janeiro,SMUL,SANTANA/ TUCURUVI,2018.00.553-00,2002-0.249.324-1,RECONSIDERACAO DE DESPACHO DE ALVARA DE EXECUC...,22/10/2002,067.225.0104-3,R202,ZCP-B/008 / EETU,Z3-013.,TUCURUVI,NaN,SOLIDEZ EMPREENDIMENTOS IMOBILIARIOS LTDA,NaN,AV GUAPIRA 291,12/01/2018,ALEXANDRE MENDONCA MILLEU,NaN,ALEXANDRE MENDONCA MILLEU,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Janeiro,SMUL/ASSEC-CEUSO,CIDADE ADEMAR,2018.01.426-00,2013-0.031.633-7,RECONSIDERACAO DE DESPACHO DE CERTIFICADO DE R...,01/02/2013,120.158.0055-6,NR3,ZCP-A/005,Z2.,VL MARARI,NaN,CONDOMÍNIO EDIFÍCIO CANAÃ,"1.110,00",R HORACIO ALVES DA COSTA 00051,31/01/2018,ARLINDO GOMES NETO,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Janeiro,SMUL/ASSEC-CEUSO,MOOCA,2018.00.588-00,1994-0.057.929-2,RECONSIDERACAO DE DESPACHO DE CERTIFICADO DE R...,31/10/1994,002.082.0031-2,C2.1,NaN,Z3-242.,BRAZ,NaN,ELIAQUIM LIMA SA,"230,67",AV RANGEL PESTANA 1321 1321A,15/01/2018,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Janeiro,SMUL/ASSEC-CEUSO,PENHA,2018.01.368-00,2012-0.238.622-5,RECONSIDERACAO DE DESPACHO DE ALVARA DE APROVA...,20/08/2012,058.021.0054-0,NR1-02,NaN,,VILA ESPERANÇA,"72,86",NATANAEL BATISTA DE ARAUJO,"183,63",AV PADRES OLIVETANOS 00692,30/01/2018,CARLOS ALBERTO GANDRA ZARA,NaN,CARLOS ALBERTO GANDRA ZARA,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Janeiro,SMUL/ASSEC-CEUSO,SE,2018.01.358-00,2008-0.155.328-4,RECONSIDERACAO DE DESPACHO DE CERTIFICADO DE R...,26/05/2008,033.033.0024-0,NR2-08,NaN,,ACLIMACAO,NaN,IGREJA CRISTA PAULISTANA,"742,00",R PIRES DA MOTA 00110,30/01/2018,GIOVANA MORELI AVANCINI,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Janeiro,SMUL/COMIN/DCIMP,ARICANDUVA/ FORMOSA/ CARRAO,2018.01.292-00,2016-0.212.311-6,ALVARA DE APROVACAO E EXECUCAO DE REFORMA,19/09/2016,056.156.0025-1,NR2-02,ZC,ZONA MISTA DE ALTA DENSIDADE - B.,VILA CARRÃO,"192,90",BDM PARTICIPACOES E EMPREENDIMENTOS,"2.129,36",AV CONSELHEIRO CARRAO 000700,29/01/2018,CARLOS ROBERTO CHICON JUNIOR,NaN,CARLOS ROBERTO CHICON JUNIOR,NaN,B: 1 P: 0 U: 0,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Janeiro,SMUL/COMIN/DCIMP,FREGUESIA/ BRASILANDIA,2017.16.931-01,2016-0.039.873-8,APOSTILAMENTO DE AUTO DE REGULARIZACAO,18/02/2016,307.042.0035-7,N

In [22]:
def df_com_col(col, df = df):
    
    df = df[df[col].notnull()].dropna(axis = 1, how = 'all')
    
    return df

In [23]:
df_com_col('zona de uso').keys()

KeyError: 'zona de uso'

In [ ]:
df_com_col('zona de uso anterior')

In [ ]:
df_com_col('ident')